# Key Concepts

* **Real business logic** - customer lookup, order tracking, refunds
* **Intelligent tool selection** - agent decides which tool to use based on customer request
* **Structured data handling** - tool results are structured data (JSON)
* **Natural language translation** - agent translates technical results into friendly responses


In [41]:
%pip install anthropic python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [42]:
# Cell 1: Setup with LiteLLM and .env
import os
from anthropic import Anthropic
from dotenv import load_dotenv
import json
import sqlite3
from datetime import datetime
from typing import Dict, Any

# Load environment variables from .env file
load_dotenv()

# Initialize Anthropic client (points to LiteLLM)
client = Anthropic(
    base_url=os.environ.get("ANTHROPIC_BASE_URL"),
    api_key=os.environ.get("ANTHROPIC_AUTH_TOKEN")
)

MODEL = os.environ.get("ANTHROPIC_MODEL", "claude-sonnet-4-20250514")

print("✅ Using Claude via LiteLLM")
print(f"🔗 Endpoint: {os.environ.get('ANTHROPIC_BASE_URL')}")
print(f"🤖 Model: {MODEL}")
print()

✅ Using Claude via LiteLLM
🔗 Endpoint: https://litellm.tikalk.dev
🤖 Model: claude-sonnet-4-20250514



In [37]:
# Customer Support Agent with Claude
from db_helper import process_refund, get_customer_orders, check_order_status
from db_tools import tools

def customer_support_agent(user_message: str):
    """A simple customer support agent using Claude via LiteLLM"""
    
    messages = [
        {
            "role": "user",
            "content": user_message
        }
    ]
    
    system_prompt = """You are a helpful customer support agent. You can:
                        - Look up customer information by customer ID
                        - Check order status for specific orders
                        - View all orders for a customer
                        - Process refunds for delivered orders
                        
                        Be polite, professional, and helpful. Always verify information before taking actions.
                        If you need to look up a customer's orders, ask for their customer ID first."""
    
    print(f"👤 Customer: {user_message}\n")
    
    # First API call
    response = client.messages.create(
        model=MODEL,
        max_tokens=1024,
        system=system_prompt,
        messages=messages,
        tools=tools
    )
    
    # Process response
    while response.stop_reason == "tool_use":
        # Add assistant's response to messages
        messages.append({
            "role": "assistant",
            "content": response.content
        })
        
        # Execute tool calls
        tool_results = []
        for content_block in response.content:
            if content_block.type == "tool_use":
                tool_name = content_block.name
                tool_input = content_block.input
                
                print(f"🔍 Agent checking: {tool_name}({tool_input})")
                
                # Execute the appropriate function
                if tool_name == "lookup_customer":
                    result = lookup_customer(**tool_input)
                elif tool_name == "check_order_status":
                    result = check_order_status(**tool_input)
                elif tool_name == "get_customer_orders":
                    result = get_customer_orders(**tool_input)
                elif tool_name == "process_refund":
                    result = process_refund(**tool_input)
                else:
                    result = {"error": "Unknown function"}
                
                print(f"📊 Database returned: {json.dumps(result, indent=2)}\n")
                
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": content_block.id,
                    "content": json.dumps(result)
                })
        
        # Add tool results to messages
        messages.append({
            "role": "user",
            "content": tool_results
        })
        
        # Get next response
        response = client.messages.create(
            model=MODEL,
            max_tokens=1024,
            system=system_prompt,
            messages=messages,
            tools=tools
        )
    
    # Extract final text response
    final_message = ""
    for content_block in response.content:
        if hasattr(content_block, "text"):
            final_message += content_block.text
    
    print(f"🤖 Agent: {final_message}")
    return final_message

In [43]:
# Test - Simple order lookup
customer_support_agent("Hi, can you check the status of order ORD12345?")

👤 Customer: Hi, can you check the status of order ORD12345?

🔍 Agent checking: check_order_status({'order_id': 'ORD12345'})
📊 Database returned: {
  "status": "found",
  "order": {
    "order_id": "ORD12345",
    "customer_id": "CUST001",
    "customer_name": "Sarah Johnson",
    "status": "shipped",
    "items": "Laptop, Mouse",
    "total": 1299.99,
    "tracking": "TRK789456123",
    "estimated_delivery": "2025-01-25"
  }
}

🤖 Agent: I found your order! Here are the details for order ORD12345:

**Order Status:** Shipped ✅

**Customer:** Sarah Johnson (CUST001)
**Items:** Laptop, Mouse
**Total:** $1,299.99
**Tracking Number:** TRK789456123
**Estimated Delivery:** January 25, 2025

Your order is currently on its way to you. You can track its progress using the tracking number TRK789456123 with the carrier. Is there anything else I can help you with regarding this order?


'I found your order! Here are the details for order ORD12345:\n\n**Order Status:** Shipped ✅\n\n**Customer:** Sarah Johnson (CUST001)\n**Items:** Laptop, Mouse\n**Total:** $1,299.99\n**Tracking Number:** TRK789456123\n**Estimated Delivery:** January 25, 2025\n\nYour order is currently on its way to you. You can track its progress using the tracking number TRK789456123 with the carrier. Is there anything else I can help you with regarding this order?'

In [44]:
# Test - Customer with all orders
customer_support_agent("My customer ID is CUST001. Can you show me all my orders?")

👤 Customer: My customer ID is CUST001. Can you show me all my orders?

🔍 Agent checking: get_customer_orders({'customer_id': 'CUST001'})
📊 Database returned: {
  "status": "success",
  "customer_name": "Sarah Johnson",
  "total_orders": 2,
  "orders": [
    {
      "order_id": "ORD12345",
      "status": "shipped",
      "items": "Laptop, Mouse",
      "total": 1299.99,
      "estimated_delivery": "2025-01-25"
    },
    {
      "order_id": "ORD12348",
      "status": "delivered",
      "items": "Monitor",
      "total": 399.99,
      "estimated_delivery": "2025-01-15"
    }
  ]
}

🤖 Agent: Hi Sarah Johnson! I found your order history for customer ID CUST001. Here are all your orders:

**Order 1:**
- Order ID: ORD12345
- Status: Shipped
- Items: Laptop, Mouse
- Total: $1,299.99
- Estimated Delivery: January 25, 2025

**Order 2:**
- Order ID: ORD12348
- Status: Delivered
- Items: Monitor
- Total: $399.99
- Estimated Delivery: January 15, 2025

You have a total of 2 orders on file. Is th

"Hi Sarah Johnson! I found your order history for customer ID CUST001. Here are all your orders:\n\n**Order 1:**\n- Order ID: ORD12345\n- Status: Shipped\n- Items: Laptop, Mouse\n- Total: $1,299.99\n- Estimated Delivery: January 25, 2025\n\n**Order 2:**\n- Order ID: ORD12348\n- Status: Delivered\n- Items: Monitor\n- Total: $399.99\n- Estimated Delivery: January 15, 2025\n\nYou have a total of 2 orders on file. Is there anything specific you'd like to know about either of these orders, or do you need any assistance with them?"

In [40]:
# Test - Refund request
customer_support_agent("I want to return order ORD12347. The product didn't match the description.")


👤 Customer: I want to return order ORD12347. The product didn't match the description.

🔍 Agent checking: check_order_status({'order_id': 'ORD12347'})
📊 Database returned: {
  "status": "found",
  "order": {
    "order_id": "ORD12347",
    "customer_id": "CUST003",
    "customer_name": "Emma Williams",
    "status": "refunded",
    "items": "Keyboard, Webcam",
    "total": 249.99,
    "tracking": "TRK789456124",
    "estimated_delivery": "2025-01-20"
  }
}

🤖 Agent: I see that order ORD12347 has already been refunded. This order for a Keyboard and Webcam totaling $249.99 belonging to Emma Williams is showing a status of "refunded" in our system.

It appears this order has already been processed for a refund. You should have received or will receive the refund according to your original payment method. If you have any concerns about the refund processing time or if you haven't received it yet, please let me know and I can help you with more information about the refund timeline.

Is the

'I see that order ORD12347 has already been refunded. This order for a Keyboard and Webcam totaling $249.99 belonging to Emma Williams is showing a status of "refunded" in our system.\n\nIt appears this order has already been processed for a refund. You should have received or will receive the refund according to your original payment method. If you have any concerns about the refund processing time or if you haven\'t received it yet, please let me know and I can help you with more information about the refund timeline.\n\nIs there anything else I can help you with regarding this order or any other concerns?'

In [45]:
# Test - Complex multi-step
customer_support_agent("Hi, I'm customer CUST003. Show me my orders and process a refund for any delivered items.")

👤 Customer: Hi, I'm customer CUST003. Show me my orders and process a refund for any delivered items.

🔍 Agent checking: lookup_customer({'customer_id': 'CUST003'})
📊 Database returned: {
  "status": "found",
  "customer": {
    "customer_id": "CUST003",
    "name": "Emma Williams",
    "email": "emma.w@email.com",
    "tier": "premium",
    "balance": -25.0
  }
}

🔍 Agent checking: get_customer_orders({'customer_id': 'CUST003'})
📊 Database returned: {
  "status": "success",
  "customer_name": "Emma Williams",
  "total_orders": 1,
  "orders": [
    {
      "order_id": "ORD12347",
      "status": "refunded",
      "items": "Keyboard, Webcam",
      "total": 249.99,
      "estimated_delivery": "2025-01-20"
    }
  ]
}

🤖 Agent: Hello Emma! I can see your account information and orders. You currently have 1 order in our system:

**Order ORD12347** - Status: **Refunded**
- Items: Keyboard, Webcam
- Total: $249.99
- Estimated Delivery: January 20, 2025

I can see that your order ORD12347 ha

'Hello Emma! I can see your account information and orders. You currently have 1 order in our system:\n\n**Order ORD12347** - Status: **Refunded**\n- Items: Keyboard, Webcam\n- Total: $249.99\n- Estimated Delivery: January 20, 2025\n\nI can see that your order ORD12347 has already been refunded, so there are no delivered items that need processing for refund at this time. \n\nIs there anything else I can help you with regarding your account or orders?'

In [ ]:
# helper - explore database
from db_helper import explore_database

explore_database()

In [ ]:
# Clean up (optional - for resetting)
from db_helper import reset_database

reset_database()